## Validação dos dados EOG

Neste notebook está incluído os seguintes passos:
- Aplicação de características;
- Criação do vetor de características;
- Normalização de dados;
- Seleção de características;
- Classificação dos dados.

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em nosso caso de EOG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

#### Domínio do tempo

1. Willison Amplitude (WAMP): 

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) $
    
    > $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn(x_i) $
    
    > $ sgn(x) = \begin{cases} 1 & \text{if } x_i * x_{i+1} \leq 0 \\ 0 & \text{otherwise} \end{cases} $

#### Domínio da frequência

1. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

2. Mean Frequency (FMN)

    > $\sum_{j=1}^{M}f_j PSD_j / \sum_{j=1}^{M}PSD_j$
    
    > $ f_j = j * SampleRate / 2 * M $

3. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{j=1}^{M}A_j $
    
    > $ A_j = Amplitude\ do\ espectro\ j $

4. Modified Frequency Mean (MMNF)

    > $ \sum_{j=1}^{M}f_jAj / \sum_{j=1}^{M}Aj $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Tarefa 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

#### Domínio do tempo

1. Willison Amplitude (WAMP): é o número de contagens para cada mudançais the number of counts for each change in the EMG signal
amplitude that exceeds a predefined threshold where threshold value is 10 mV.

2. Variance of EMG (VAR): expressa o poder do sinal EMG como caracteristica utilizavel.

3. Root Mean Square (RMS): RMS representa caracteristicas no dominio do tempo baseado na amplitude do sinal como valor absoluto médio, inclinação média do valor absoluto, e variação.

4. Waveform Length (WL): é o cumprimento cumulativo da onda durante o segmento de tempo. WL é relacionado a amplitude, frequência e tempo da onda.

5. Zero Crossing (ZC): é o número de vezes que os sinais EMG passam de 0. O valor limit é 20 mV. Essas características fornecem uma estimativa aproximada das propriedades no dominio da frequência.

#### Domínio da frequência

1. Median Frequency (FMD): é a frequência na qual o espectro é dividido em duas regiões com potências iguais.

2. Mean Frequency (FMN): é a frequência média.

3. Modified Median Frequency (MMDF): é bem similar ao método FMD, mas é baseado no espectro de amplitude, não no PSD.

4. Modified Frequency Mean (MMNF): é a frequência média baseada no espectro da amplitude diferente do FMN.

#### Aplicando as características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características `VAR` & `RMS` (domínio do tempo) e `FDM` & `MMDF` (domínio da frequência).

In [581]:
# Libraries
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [582]:
# Load
data_time = np.load("dataset/data_time.npy")
data_freq = np.load("dataset/data_freq.npy")
data_time.shape, data_freq.shape

((28, 2, 4, 64), (28, 2, 7, 33))

**Tarefa 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características além das apresentadas, citando as fontes de tais características.


#### Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

#### Implementação do vetor

In [583]:
# Funções auxiliares das funções de extração de características
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2

def fj(w):
    sample_rate = np.arange(1, w.shape[-1]+1) * 200
    return sample_rate / 2 * w.shape[-1]

# Funções de extração de características
def wamp(time, threshold):
    return np.sum(np.abs(np.diff(time)) > threshold, axis=-1)

def var(time):
    return np.sum(time ** 2, axis=-1) / (np.prod(time.shape) - 1)

def rms(time):
    return np.sqrt(np.sum(np.abs(time) ** 2, axis=-1) / (np.prod(time.shape) - 1))

def wl(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)

def zc(x):
    return np.count_nonzero(np.diff(np.sign(x), axis=-1) != 0, axis=-1)

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def fmn(w):
    return np.sum(fj(w) * PSD(w)) / fmd(w)*2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

def mmnf(w):
    return np.sum(fj(w) * np.abs(w),  axis=-1) / mmdf(w)*2

In [584]:
# Aplicando características
data_wamp = wamp(data_time, 1)
data_var = var(data_time)
data_rms = rms(data_time)
data_wl = wl(data_time)
data_zc = zc(data_time)

data_fmd = fmd(data_freq)
data_fmn = fmn(data_freq)
data_mmdf = mmdf(data_freq)
data_mmnf = mmnf(data_freq)

data_wamp.shape, data_var.shape, data_rms.shape, data_wl.shape, data_zc.shape, data_fmd.shape, data_fmn.shape, data_mmdf.shape, data_mmnf.shape

((28, 2, 4),
 (28, 2, 4),
 (28, 2, 4),
 (28, 2, 4),
 (28, 2, 4),
 (28, 2, 7),
 (28, 2, 7),
 (28, 2, 7),
 (28, 2, 7))

In [585]:
# União do vetor de características inicial
size = data_wamp.shape[2]
print(size)

features = np.array([
    data_wamp, data_var, data_rms, data_wl, data_zc,
    data_fmd[:,:,:size], data_fmn[:,:,:size], data_mmdf[:,:,:size], data_mmnf[:,:,:size]
])

# Organização das dimensões
features = features.transpose(1, 3, 0, 2)

# Criar vetor de características definitivo
features = features.reshape(features.shape[0] * features.shape[1],
                            features.shape[2] * features.shape[3])

features.shape

4


(112, 18)

*Tarefa 3*: Realização da normalização dos dados utilizando ferramentas já conhecidas

In [586]:
labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]

# criação do vetor de labels final
labels = np.repeat(labels_num, int(features.shape[0] / len(labels_num)))
len(labels)

112

In [587]:
# Aplicando normalização
data_final = StandardScaler().fit_transform(features)
labels_final = LabelEncoder().fit_transform(labels)

data_final = MinMaxScaler().fit_transform(data_final)
data_final.shape, labels_final.shape

((112, 18), (112,))

*Tarefa 4*: Realização da seleção de características, utilizando ferramentas já conhecidas
*Tarefa 5*: Realização da classificação utilizando `SVM`.

In [588]:
for k in range(1, data_final.shape[1]+1):
    # Aplicando seleção de características
    new_data = SelectKBest(f_classif, k=k).fit_transform(data_final, labels_final)

    # Aplicando a classificação
    X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(new_data, labels_final, test_size=0.2, random_state=0)
    # print(X_train_full.shape, labels_final.shape)
    svm = SVC(kernel='linear', C=10, probability=True).fit(X_train_full, y_train_full)
    y_full_pred = svm.predict(X_test_full)

    print("K: %d | Accuracy: %.2f %%" % (k, accuracy_score(y_test_full, y_full_pred) * 100))

K: 1 | Accuracy: 43.48 %
K: 2 | Accuracy: 52.17 %
K: 3 | Accuracy: 56.52 %
K: 4 | Accuracy: 56.52 %
K: 5 | Accuracy: 78.26 %
K: 6 | Accuracy: 69.57 %
K: 7 | Accuracy: 69.57 %
K: 8 | Accuracy: 82.61 %
K: 9 | Accuracy: 78.26 %
K: 10 | Accuracy: 65.22 %
K: 11 | Accuracy: 69.57 %
K: 12 | Accuracy: 69.57 %
K: 13 | Accuracy: 73.91 %
K: 14 | Accuracy: 69.57 %
K: 15 | Accuracy: 60.87 %
K: 16 | Accuracy: 65.22 %
K: 17 | Accuracy: 69.57 %
K: 18 | Accuracy: 65.22 %
